In [42]:
import os
import operator
import json

with open(os.getcwd() + "/scripts/" + "Thor-Ragnarok_script.txt", "r") as s:
    thor = s.read()

In [43]:
print(thor[:300])
print("\n" + "*"*50 + "\n")
thor[:2000]




                               THOR: RAGNAROK




                                 Written by

             Eric Pearson and Craig Kyle &amp; Christopher L. Yost







 1    OMITTED                                                         1

A2   OMITTED                                           

**************************************************



"\n\n\n                               THOR: RAGNAROK\n\n\n\n\n                                 Written by\n\n             Eric Pearson and Craig Kyle &amp; Christopher L. Yost\n\n\n\n\n\n\n\n 1    OMITTED                                                         1\n\nA2   OMITTED                                                         A2\n\nA3   THE MARVEL LOGO. SMOLDERING, BEGINNING TO TURN ORANGE IN THE    A3\n     HEAT AS WE TILT UP TO SEE-\n\n     -FIRE.    NOTHING BUT FIRE.\n\n2    INT. TIGHT SPACE - INDETERMINATE TIME                           2\n\n     Dark and cramped. The soft red light of fire seeps through\n     iron slats. Inside this cage is a man, bound by chains.\n\n     It's THOR. His beard is long and his clothes are worn. That\n     rough, grizzled look of a man who's spent years on the road.\n\n     He awakens with a JOLT.   Looks around.\n\n                           THOR\n                 Now I know what you're thinking.\n                 Oh no! Thor's in a cage. How

In [44]:
# Gets the frecuency of appearance of spaces at the beginning
# (0, 2335) means that 2335 lines begin with 0 spaces
line_space_freq = {}
for line in thor.split("\n"):
    key = line.count(" ")
    if key not in line_space_freq:
        line_space_freq[key] = 1
    else:
        line_space_freq[key] += 1
sorted_line_space_freq = sorted(line_space_freq.items(), key=operator.itemgetter(1), reverse=True)
sorted_line_space_freq[:10]

[(0, 2335),
 (15, 437),
 (20, 426),
 (16, 375),
 (14, 318),
 (21, 313),
 (13, 241),
 (10, 229),
 (17, 228),
 (11, 227)]

0 corresponds to screen-play and randoms `""`. 1 to 5 or so corresponds also to screen-play. Below 17 corresponds to character dialogue. Above corresponds to names and scenes and stuff introductions. Thus, a space of 10 to cut is OK, at least for this movie.

In [45]:
# Counts the frequency of every unique sentence that starts with more than 10 spaces
# With the spaces before and after text removed, and also with "(some_text)" removed 
poss_char = {}
for line in thor.split("\n"):
    if line.count(" ") > 10:  # Relative low space (this condition may not even be neccessary)
        # Need to do some processing
        key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
        if key:  # To omit ""
            if key not in poss_char:
                poss_char[key] = 1
            else:
                poss_char[key] += 1

In [46]:
# Gets sentences with more than 5 occurences (potential characters)
sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
threshold = 5
chars = []
for char_tuple in sorted_poss_chars:
    if char_tuple[1] > threshold:
        chars.append(char_tuple)
    else:
        break
chars

[('THOR', 348),
 ('LOKI', 88),
 ('VALKYRIE', 71),
 ('BANNER', 64),
 ('HELA', 59),
 ('GRANDMASTER', 49),
 ('HULK', 42),
 ('KORG', 33),
 ('ODIN', 30),
 ('SKURGE', 19),
 ('HEIMDALL', 19),
 ('SURTUR', 14),
 ('SCRAPPER #142', 14),
 ('STRANGE', 13),
 ('ACTOR THOR', 12),
 ('ACTOR LOKI', 11),
 ('TOPAZ', 10),
 ('QUINJET', 9),
 ('SOOTHING VOICE', 8),
 ('DR. STRANGE', 7)]

#### **Idea**:

This threshold (5 in this case) could be tuned using IMDb! We compare with the list of main characters listed on site, get the overall accuracy for all movies for different thresholds, and choose the threshold with the highest accuracy. 

*UPDATE*: Upon much thought, we will choose the characters with the most dialogue instead. TODO: Decide on the number of characters....

Note: the `if line.count(" ") > 10:` condition may not be needed, which would be great for those scripts that do not follow this format. See below:

In [47]:
poss_char = {}
for line in thor.split("\n"):
    key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
    if key:
        if key not in poss_char:
            poss_char[key] = 1
        else:
            poss_char[key] += 1

In [48]:
sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
threshold = 5
chars = []
for char_tuple in sorted_poss_chars:
    if char_tuple[1] > threshold:
        chars.append(char_tuple)
    else:
        break
chars

[('THOR', 357),
 ('LOKI', 88),
 ('VALKYRIE', 71),
 ('BANNER', 67),
 ('HELA', 59),
 ('GRANDMASTER', 49),
 ('HULK', 42),
 ('KORG', 33),
 ('ODIN', 30),
 ('SKURGE', 19),
 ('HEIMDALL', 19),
 ('STRANGE', 17),
 ('SURTUR', 14),
 ('SCRAPPER #142', 14),
 ('ACTOR THOR', 12),
 ('ACTOR LOKI', 11),
 ('TOPAZ', 10),
 ('QUINJET', 9),
 ('SOOTHING VOICE', 8),
 ('What?', 8),
 ('DR. STRANGE', 7)]

The result is pretty much the same, although `'What?'` is apparantly a character. This could be avoided by `.isupper()` or maybe checking for punct sings. The `x` in `if line.count(" ") > x` could also be tuned.  

Actually, I found this to be problematic, so I set it to `-1`.

In [51]:
def get_characters(script, threshold_char=5, threshold_space=-1):
    
    with open(os.getcwd() + "/scripts/" + script, "r") as s:
        script = s.read()
        
    poss_char = {}
    for line in script.split("\n"):
        if line.count(" ") > threshold_space:
            key = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ").replace("\t", "").strip(" ")
            if key not in poss_char:
                poss_char[key] = 1
            else:
                poss_char[key] += 1
                        
    sorted_poss_chars = sorted(poss_char.items(), key=operator.itemgetter(1), reverse=True)
    chars = []
    for char_tuple in sorted_poss_chars:
        if char_tuple[1] > threshold_char:
            chars.append(char_tuple)
        else:
            break
            
    return [char[0] for char in chars if char[0] != "" and "-" not in char[0] and "." not in char[0] and ":" not in char[0] and "?" not in char[0]]


get_characters("Thor-Ragnarok_script.txt")

['THOR',
 'LOKI',
 'VALKYRIE',
 'BANNER',
 'HELA',
 'GRANDMASTER',
 'HULK',
 'KORG',
 'ODIN',
 'SKURGE',
 'HEIMDALL',
 'STRANGE',
 'SURTUR',
 'SCRAPPER #142',
 'ACTOR THOR',
 'ACTOR LOKI',
 'TOPAZ',
 'QUINJET',
 'SOOTHING VOICE']

Improved version of getting the characters dialogues, and also ranks character by dialogue length

In [85]:
#Alternate Approach which works equally well.
"""
script = open('NLP-Movie_Scripts/scripts/'+file,'r')
for line in script:
    temp = line.strip()
    if 'thor'.upper() in temp:
        if (len(temp.split(' '))  <= 2) & (temp.split(' ')[0]=='thor'.upper()):
            print(temp)
"""
movies_diag = {}
for movie in os.listdir(os.getcwd() + "/scripts/"):
    movies_diag[movie] = {}
    if movie[-4:] == ".txt":
        with open(os.getcwd() + "/scripts/" + movie, "r") as s:
            script = s.read()
        #create a string containing character names seperated by commas
        characters = ','.join([','.join(name.split(' ')) for name in get_characters(movie) if len(name.split(' ')) <3])
        # data structure to store the dialogues
        dialogues = {}
        for ch in characters.split(','):
            if 'Mr.' in ch or '-' in ch:
                continue
            else:
                name = ch
            dialogue = '' 
            flag = 0
            count = 0
            for line in script.split("\n"):
                temp = line.strip(" ").replace(line[line.find("("):line.find(")") + 1], "").strip(" ")
                if temp.isupper():  # If Character Intro
                    if name.upper() == temp:
                        flag = 1
                        count += 1
                        dialogue += '\n['+str(count)+']'
                        continue
                    else:  # If it's another character
                        flag = 0
                        continue
                if flag:
                    if flag == 1:
                        if line.strip(): # Accounts for first lines == "" or == "     "
                            if line.lstrip(" ")[0] == "(" or line.rstrip(" ")[-1] == ")":  # To still get the dialogue
                                ident_level_first = None  # when the first line below a character
                                flag = 0  # intro is something like "(looking around with a grin)"
                            else:  # which describes how the character is acting but it is not his dialogue
                                ident_level_first = len(line) - len(line.lstrip(" "))
                        else:
                            ident_level_first = None
                            flag = 0
                    ident_level = len(line) - len(line.lstrip(" "))
                    if ident_level_first:
                        if ident_level == ident_level_first:
                            dialogue += temp + " "
                        else:
                            continue
                    else:
                        pass
                    flag += 1
            dialogues[name] = dialogue
        movies_diag[movie]['dialogues'] = dialogues

In [88]:
dict_lengths = {}
for key, value in movies_diag["Thor-Ragnarok_script.txt"]["dialogues"].items():
    a = value.replace("\n", "").replace(" ", "")
    i = 1
    while True:
        before = len(a)
        a = a.replace("[" + str(i) + "]", "")
        i += 1
        if len(a) == before:
            break
    dict_lengths[key] = len(a)

sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)

[('THOR', 13749),
 ('LOKI', 3371),
 ('GRANDMASTER', 3294),
 ('HELA', 2820),
 ('VALKYRIE', 2534),
 ('KORG', 2430),
 ('BANNER', 1891),
 ('ODIN', 1389),
 ('SKURGE', 1075),
 ('SURTUR', 666),
 ('HULK', 584),
 ('HEIMDALL', 510),
 ('STRANGE', 332),
 ('TOPAZ', 267),
 ('QUINJET', 160),
 ('SCRAPPER', 22),
 ('#142', 0),
 ('ACTOR', 0),
 ('SOOTHING', 0),
 ('VOICE', 0)]

In [89]:
[i[0] for i in sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)[:5]]

['THOR', 'LOKI', 'GRANDMASTER', 'HELA', 'VALKYRIE']

## Comparing with IMDb

TODO: Decide on `n_characters_choose`

In [82]:
# Taken from `script_processor.py`

import omdb
import requests
from bs4 import BeautifulSoup

files = os.listdir(os.getcwd() + '/scripts/')

#Create a a dict where each item contains information on a movie in dictionary format
movies = {}
for each in files:
    movie = {}
    if '-The_script.txt' in each:
        movies['The '+each.replace('-The_script.txt','').replace('-', ' ')[:-1]] = {'filename':each}
    else:
        movies[each.replace('_script.txt','').replace('-', ' ')] = {'filename':each}

#fetch imdb data of a movie using OMDB api, please read OMDB doc for further details
def getOmdbData(movie):
    api_key = 'eb6547ff'
    omdb.set_default('apikey', api_key)
    return omdb.get(title=movie, fullplot=True, tomatoes=True)

def prepURL(imdb_id):
	#prepare the url from which we will scrape the cast data
    url = 'https://www.imdb.com/title/'+imdb_id#+'/fullcredits?ref_=tt_cl_sm#cast'
    return url

def getCharacters(url):
	#Only first imdb page of the cast members are considered. 
    website_url = requests.get(url).text
    soup = BeautifulSoup(website_url, "lxml")
    #upon inspecting the html code, we see that the cast is structured in <table> tags
    table = soup.find('table',{'class':'cast_list'}) 
    #<tr> is a tag for table row [tr]
    cast = table.find_all('tr') 
    characters = []
    for each in cast:
        temp = each.find_all('td')[-1].text.strip().replace('\n','').replace('  ','')
        if 'uncredited' in temp:
            characters.append(temp.split('(')[0][:-1])
        else:
            characters.append(temp)
    characters = list(set(characters))
    return characters

for movie in movies.keys():
	#for the given movie name we fetch the imdb data
    x = getOmdbData(movie)
    try:
        movies[movie]['imdb_id'] = x['imdb_id']
    except:
        continue

#We need to delete the movie's on which there isn't much data available
delete = []
for movie in movies.keys():
    try:
        url = prepURL(movies[movie]['imdb_id'])
        characters = getCharacters(url)
        movies[movie]['imdb_url'] = url
        movies[movie]['characters'] = characters
    except:
        delete.append(movie)

#Delete the movies on which we don't have necessary data on
for movie in delete:
    del movies[movie]

#Clean the data further
for movie in movies.keys():
    if 'Cast overview, first billed only:' in movies[movie]['characters']:
        movies[movie]['characters'].remove('Cast overview, first billed only:')

In [92]:
overall_acc = {}
problematic_scripts = []
print_one = True
n_characters_choose = 5
accs = []
for script in os.listdir(os.getcwd() + "/scripts/"):
    if script[-4:] == ".txt":
        # Gets the characters using our function
        chars = get_characters(script, 5)
        if len(chars) == 0:
            problematic_scripts.append(script)
            brr = True
        else:
            # Gets the top n_characters_choose characters with the most dialogue
            dict_lengths = {}
            for key, value in movies_diag[script]["dialogues"].items():
                a = value.replace("\n", "").replace(" ", "")
                i = 1
                while True:
                    before = len(a)
                    a = a.replace("[" + str(i) + "]", "")
                    i += 1
                    if len(a) == before:
                        break
                dict_lengths[key] = len(a)
            chars_sorted_len_diag = sorted(dict_lengths.items(), key=operator.itemgetter(1), reverse=True)
            chars = [i[0] for i in chars_sorted_len_diag[:n_characters_choose]]
            #Gets list of characters from IMDB
            if '-The_script.txt' in script:
                key = 'The '+script.replace('-The_script.txt','').replace('-', ' ')[:-1]
            else:
                key = script.replace('_script.txt','').replace('-', ' ')
            try:
                chars_imdb = movies[key]["characters"]
            except KeyError:
                pass
            else:
                #Turns both to UPPERCASE (in case chars is not, and IMDB will most likely not be)
                chars_upper = [char.upper() for char in chars]
                chars_imdb_upper = [char.upper() for char in chars_imdb]
                # Compares one by one and add 1 to `matches` if at least one word of one is equal to one word of the other?
                # This condition could probably be improved, but we need to do some visual testing, so we probably want to
                # keep a list with the names of the characters that matched
                matches = 0
                br = False
                for char in chars_upper:
                    for word in char.split(" "):
                        for imdb_char in chars_imdb_upper:
                            if word in imdb_char:
                                matches += 1
                                br = True
                                break
                        if br:
                            break
                accs.append(matches/len(chars))  # Proportion of identified characters that were in IMDb's main cast
                if accs[-1] < 0.1 and print_one:
                    print(script)
                    print_one = False

print(sum(accs)/len(accs))

Godzilla_script.txt
0.8106508875739663


In [61]:
movies["Godzilla"]["characters"]

['Vivienne Graham',
 'Joe Brody',
 'Captain Russell Hampton',
 'Jump Master',
 'Admiral William Stenz',
 'Sergeant Tre Morales',
 'HALO Jumper',
 'Young Ford',
 'Elle Brody',
 'Sandra Brody',
 'Dr. Ishiro Serizawa',
 'Sam Brody',
 'Bomb Tracker',
 'Lieutenant Commander Marcus Waltz',
 'Ford Brody']

In [62]:
get_characters("Godzilla_script.txt", 5)

['NICK',
 'AUDREY',
 'RAYMOND',
 'MAJOR HICKS',
 'ANIMAL',
 'CAIMAN',
 'LUCY',
 'MAYOR',
 'CUT TO:',
 'ELSIE',
 "SERGEANT O'NEAL",
 'GENERAL ANDERSON',
 'CAPTAIN',
 'JEAN-CLAUDE',
 'ENSIGN']

I think it's actually [Godzilla from 1998](https://www.imdb.com/title/tt0120685/)... We may be better off using my function and we can avoid issues such as this one. For a better evaluation, I think Sean was using the release date on our `.csv` to get the correct movie, when there are these discrepancies.